## import library and load Data 

In [3]:
from  pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import countDistinct
from pyspark.sql import functions as f

In [4]:
spark = SparkSession.builder.appName("My_Spark_Project")\
.config("spark.memory.offHeap. enabled", "true") . config("spark.memory.offHeap. size", "10g") \
.getOrCreate()

* show customers dataset


In [5]:
file_path = r"C:\Users\abdel\OneDrive\Desktop\Sub\BigData\Csv. files\customers.csv"
# Read the CSV file into a DataFrame
df_customers = spark.read.csv(file_path, header=True, inferSchema=True)
df_customers .show()


+-----------+-------------------+-----------+---+--------------------+--------+-------------------+--------------------+---------+
|customer_id|      customer_name|     gender|age|        home_address|zip_code|               city|               state|  country|
+-----------+-------------------+-----------+---+--------------------+--------+-------------------+--------------------+---------+
|          1|      Leanna Busson|     Female| 30|8606 Victoria Ter...|    5464|      Johnstonhaven|  Northern Territory|Australia|
|          2|Zabrina Harrowsmith|Genderfluid| 69|8327 Kirlin Summi...|    8223|    New Zacharyfort|     South Australia|Australia|
|          3|    Shina Dullaghan| Polygender| 59|269 Gemma SummitS...|    5661|           Aliburgh|Australian Capita...|Australia|
|          4|      Hewet McVitie|   Bigender| 67|743 Bailey GroveS...|    1729|  South Justinhaven|          Queensland|Australia|
|          5|     Rubia Ashleigh| Polygender| 30|48 Hyatt ManorSui...|    4032|    

In [6]:
df_customers .printSchema()
print("Count of dataframe:",df_customers.count())

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- home_address: string (nullable = true)
 |-- zip_code: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)

Count of dataframe: 1000


In [7]:
df_customers.describe().show()

+-------+-----------------+--------------+----------+------------------+--------------------+-----------------+---------+--------------------+---------+
|summary|      customer_id| customer_name|    gender|               age|        home_address|         zip_code|     city|               state|  country|
+-------+-----------------+--------------+----------+------------------+--------------------+-----------------+---------+--------------------+---------+
|  count|             1000|          1000|      1000|              1000|                1000|             1000|     1000|                1000|     1000|
|   mean|            500.5|          null|      null|             49.86|                null|         5004.872|     null|                null|     null|
| stddev|288.8194360957494|          null|      null|17.647828360618387|                null|2884.497332027621|     null|                null|     null|
|    min|                1|Abbot Rickaert|   Agender|                20|00 Fadel C

In [8]:
df_customers.describe(["age", "zip_code"]).show()

+-------+------------------+-----------------+
|summary|               age|         zip_code|
+-------+------------------+-----------------+
|  count|              1000|             1000|
|   mean|             49.86|         5004.872|
| stddev|17.647828360618387|2884.497332027621|
|    min|                20|                2|
|    max|                80|             9998|
+-------+------------------+-----------------+



# EDA

#### Cleanning data

In [9]:
df_customers.show()

+-----------+-------------------+-----------+---+--------------------+--------+-------------------+--------------------+---------+
|customer_id|      customer_name|     gender|age|        home_address|zip_code|               city|               state|  country|
+-----------+-------------------+-----------+---+--------------------+--------+-------------------+--------------------+---------+
|          1|      Leanna Busson|     Female| 30|8606 Victoria Ter...|    5464|      Johnstonhaven|  Northern Territory|Australia|
|          2|Zabrina Harrowsmith|Genderfluid| 69|8327 Kirlin Summi...|    8223|    New Zacharyfort|     South Australia|Australia|
|          3|    Shina Dullaghan| Polygender| 59|269 Gemma SummitS...|    5661|           Aliburgh|Australian Capita...|Australia|
|          4|      Hewet McVitie|   Bigender| 67|743 Bailey GroveS...|    1729|  South Justinhaven|          Queensland|Australia|
|          5|     Rubia Ashleigh| Polygender| 30|48 Hyatt ManorSui...|    4032|    

> here we will check if there null values from our dataset 

In [10]:
df_customers=df_customers.dropna()
df_customers.count()

1000

In [11]:
from pyspark.sql.functions import when

# Replace values in 'gender' column directly
df_customers = df_customers.withColumn(
    'gender',
    when(df_customers['gender'].isin(['Male', 'Female']), df_customers['gender']).otherwise('Other')
)

# Show the updated DataFrame
df_customers.show()


+-----------+-------------------+------+---+--------------------+--------+-------------------+--------------------+---------+
|customer_id|      customer_name|gender|age|        home_address|zip_code|               city|               state|  country|
+-----------+-------------------+------+---+--------------------+--------+-------------------+--------------------+---------+
|          1|      Leanna Busson|Female| 30|8606 Victoria Ter...|    5464|      Johnstonhaven|  Northern Territory|Australia|
|          2|Zabrina Harrowsmith| Other| 69|8327 Kirlin Summi...|    8223|    New Zacharyfort|     South Australia|Australia|
|          3|    Shina Dullaghan| Other| 59|269 Gemma SummitS...|    5661|           Aliburgh|Australian Capita...|Australia|
|          4|      Hewet McVitie| Other| 67|743 Bailey GroveS...|    1729|  South Justinhaven|          Queensland|Australia|
|          5|     Rubia Ashleigh| Other| 30|48 Hyatt ManorSui...|    4032|     Griffithsshire|          Queensland|Aus

> here we will check if there Duplicates or outliers in our dataset  and if there will handle it 

In [12]:
from pyspark.sql.functions import col
# Handling Duplicates
df_customers =df_customers.dropDuplicates()
# Handling Outliers 
# Assuming outliers are defined as age < 0 or age > 100
df_customers =df_customers.filter((col("age") >= 0) & (col("age") <= 100))
df_customers.show(30)

+-----------+--------------------+------+---+--------------------+--------+--------------------+--------------------+---------+
|customer_id|       customer_name|gender|age|        home_address|zip_code|                city|               state|  country|
+-----------+--------------------+------+---+--------------------+--------+--------------------+--------------------+---------+
|        152|     Panchito Wybern|Female| 79|03 Glover TrackAp...|    7845|       Christianport|Australian Capita...|Australia|
|        239|      Daria Embleton| Other| 63|39 Wilkinson Cres...|    1644|        North Sophie|Australian Capita...|Australia|
|        367|Hildagarde Dell '...| Other| 51|59 Henry PlaceSui...|     514|          Jasperside|     South Australia|Australia|
|        399|     Ailene McKeller| Other| 46|9444 Casper Ridge...|    8209|         South Logan|   Western Australia|Australia|
|        446|       Benny Songist| Other| 47|165 Hermiston Loo...|    1552|     North Lukeshire|        

In [13]:
df_customers = df_customers.drop('new_gender')
df_customers.show()

+-----------+--------------------+------+---+--------------------+--------+--------------------+--------------------+---------+
|customer_id|       customer_name|gender|age|        home_address|zip_code|                city|               state|  country|
+-----------+--------------------+------+---+--------------------+--------+--------------------+--------------------+---------+
|        152|     Panchito Wybern|Female| 79|03 Glover TrackAp...|    7845|       Christianport|Australian Capita...|Australia|
|        239|      Daria Embleton| Other| 63|39 Wilkinson Cres...|    1644|        North Sophie|Australian Capita...|Australia|
|        367|Hildagarde Dell '...| Other| 51|59 Henry PlaceSui...|     514|          Jasperside|     South Australia|Australia|
|        399|     Ailene McKeller| Other| 46|9444 Casper Ridge...|    8209|         South Logan|   Western Australia|Australia|
|        446|       Benny Songist| Other| 47|165 Hermiston Loo...|    1552|     North Lukeshire|        

In [14]:
df_customers.groupBy("gender").count().show()
df_customers.groupBy("country").count().show()

+------+-----+
|gender|count|
+------+-----+
|Female|  115|
| Other|  742|
|  Male|  143|
+------+-----+

+---------+-----+
|  country|count|
+---------+-----+
|Australia| 1000|
+---------+-----+



In [15]:
from pyspark.sql import functions as F
# Count the number of unique states
unique_states_df = df_customers.agg(F.countDistinct("state").alias("unique_states"))
unique_states_df.show()
# Count the number of unique cities
unique_cities_df = df_customers.agg(F.countDistinct("city").alias("unique_cities"))
unique_cities_df.show()

+-------------+
|unique_states|
+-------------+
|            8|
+-------------+

+-------------+
|unique_cities|
+-------------+
|          961|
+-------------+



In [16]:
from pyspark.sql import functions as F
# Calculate the average age of customers
avg_age_df = df_customers.agg(F.avg("age").alias("average_age"))
avg_age_df.show()

+-----------+
|average_age|
+-----------+
|      49.86|
+-----------+



In [17]:
df_customers = df_customers.drop('country')
df_customers.show()

+-----------+--------------------+------+---+--------------------+--------+--------------------+--------------------+
|customer_id|       customer_name|gender|age|        home_address|zip_code|                city|               state|
+-----------+--------------------+------+---+--------------------+--------+--------------------+--------------------+
|        152|     Panchito Wybern|Female| 79|03 Glover TrackAp...|    7845|       Christianport|Australian Capita...|
|        239|      Daria Embleton| Other| 63|39 Wilkinson Cres...|    1644|        North Sophie|Australian Capita...|
|        367|Hildagarde Dell '...| Other| 51|59 Henry PlaceSui...|     514|          Jasperside|     South Australia|
|        399|     Ailene McKeller| Other| 46|9444 Casper Ridge...|    8209|         South Logan|   Western Australia|
|        446|       Benny Songist| Other| 47|165 Hermiston Loo...|    1552|     North Lukeshire|            Tasmania|
|        613|   Melamie Craigmyle| Other| 78|2525 Leanno


**What are the counts of unique customers per state, along with the average age of customers in each state?**

In [18]:
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import avg
country_stats = df_customers.groupBy('state').agg(
    countDistinct('customer_id').alias('customer_count'),
    avg('age').alias('avg_age')
)

# Show the results
country_stats.show()

+--------------------+--------------+------------------+
|               state|customer_count|           avg_age|
+--------------------+--------------+------------------+
|  Northern Territory|           125|            49.168|
|     South Australia|           139| 49.86330935251799|
|            Victoria|           121| 47.83471074380165|
|   Western Australia|           124| 48.70161290322581|
|     New South Wales|           132| 52.88636363636363|
|Australian Capita...|           121|50.710743801652896|
|            Tasmania|           104| 48.59615384615385|
|          Queensland|           134|50.634328358208954|
+--------------------+--------------+------------------+





**Which state has the highest number of customers?**

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# Group by city and count the number of customers in each city
city_customer_count = df_customers.groupBy("state").agg(F.countDistinct("customer_id").alias("customer_count"))
# Sort cities by customer count in descending order
sorted_cities = city_customer_count.orderBy(F.desc("customer_count"))
sorted_cities.show()
most_customers_city = sorted_cities.select("state", "customer_count").first()
# Show the city with the most customer count
print(f"State with the most customers: {most_customers_city['state']} ({most_customers_city['customer_count']} customers)")

+--------------------+--------------+
|               state|customer_count|
+--------------------+--------------+
|     South Australia|           139|
|          Queensland|           134|
|     New South Wales|           132|
|  Northern Territory|           125|
|   Western Australia|           124|
|            Victoria|           121|
|Australian Capita...|           121|
|            Tasmania|           104|
+--------------------+--------------+

State with the most customers: South Australia (139 customers)


* show orders dataset

In [20]:
file_path = r"C:\Users\abdel\OneDrive\Desktop\Sub\BigData\Csv. files\orders.csv"
# Read the CSV file into a DataFrame
df_orders = spark.read.csv(file_path, header=True, inferSchema=True)
df_orders.show()

+--------+-----------+-------+-------------------+-------------+
|order_id|customer_id|payment|         order_date|delivery_date|
+--------+-----------+-------+-------------------+-------------+
|       1|         64|  30811|2021-08-30 00:00:00|   2021-09-24|
|       2|        473|  50490|2021-02-03 00:00:00|   2021-02-13|
|       3|        774|  46763|2021-10-08 00:00:00|   2021-11-03|
|       4|        433|  39782|2021-05-06 00:00:00|   2021-05-19|
|       5|        441|  14719|2021-03-23 00:00:00|   2021-03-24|
|       6|        800|  16197|2021-09-09 00:00:00|   2021-10-05|
|       7|        626|  37666|2021-04-05 00:00:00|   2021-04-11|
|       8|         58|  28484|2021-04-12 00:00:00|   2021-05-01|
|       9|        852|  12896|2021-05-01 00:00:00|   2021-05-11|
|      10|        659|  21922|2021-10-15 00:00:00|   2021-10-16|
|      11|        785|  36624|2021-06-15 00:00:00|   2021-06-30|
|      12|        120|  55507|2021-06-30 00:00:00|   2021-07-11|
|      13|        204|  5

In [21]:
df_orders.printSchema()
print("Count of dataframe:",df_orders.count())

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- payment: integer (nullable = true)
 |-- order_date: timestamp (nullable = true)
 |-- delivery_date: date (nullable = true)

Count of dataframe: 1000


# EDA

#### Cleanning data

In [22]:
from pyspark.sql.functions import col
# Handling Missing Values
# Drop rows with any null values
df_orders= df_orders.dropna()

# Handling Duplicates
df_orders =df_orders.dropDuplicates()
print("Count of after cleanning dataframe:",df_orders.count())
df_orders.show()

Count of after cleanning dataframe: 1000
+--------+-----------+-------+-------------------+-------------+
|order_id|customer_id|payment|         order_date|delivery_date|
+--------+-----------+-------+-------------------+-------------+
|      55|        873|  55650|2021-04-08 00:00:00|   2021-04-25|
|      66|        631|  42774|2021-08-14 00:00:00|   2021-08-22|
|     104|        366|  11454|2021-06-05 00:00:00|   2021-06-23|
|     148|        573|  14073|2021-01-14 00:00:00|   2021-01-24|
|     157|        909|  17955|2021-04-07 00:00:00|   2021-04-17|
|     513|        950|  49647|2021-05-24 00:00:00|   2021-06-04|
|     811|        764|  23731|2021-10-22 00:00:00|   2021-10-24|
|     834|        139|  27691|2021-02-23 00:00:00|   2021-03-08|
|     984|        359|  10390|2021-09-23 00:00:00|   2021-09-30|
|      99|        291|  26787|2021-09-13 00:00:00|   2021-09-25|
|     309|        390|  26229|2021-01-07 00:00:00|   2021-01-25|
|     633|        183|  30197|2021-07-02 00:00:00

In [23]:
from pyspark.sql.functions import col, to_date

# Assuming 'order_date' is your timestamp column
df_orders= df_orders.withColumn('order_date', to_date(col('order_date')))

# Show the updated schema to verify the data type change
df_orders.printSchema()
df_orders. show()

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- payment: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- delivery_date: date (nullable = true)

+--------+-----------+-------+----------+-------------+
|order_id|customer_id|payment|order_date|delivery_date|
+--------+-----------+-------+----------+-------------+
|      55|        873|  55650|2021-04-08|   2021-04-25|
|      66|        631|  42774|2021-08-14|   2021-08-22|
|     104|        366|  11454|2021-06-05|   2021-06-23|
|     148|        573|  14073|2021-01-14|   2021-01-24|
|     157|        909|  17955|2021-04-07|   2021-04-17|
|     513|        950|  49647|2021-05-24|   2021-06-04|
|     811|        764|  23731|2021-10-22|   2021-10-24|
|     834|        139|  27691|2021-02-23|   2021-03-08|
|     984|        359|  10390|2021-09-23|   2021-09-30|
|      99|        291|  26787|2021-09-13|   2021-09-25|
|     309|        390|  26229|2021-01-07|   2021-01-25|
| 

**What are the counts of unique orders and unique customers in the dataset?**

In [24]:
from pyspark.sql import functions as F

# Aggregations for order_df DataFrame
order_aggregations = df_orders.agg(
    F.countDistinct("order_id").alias("unique_orders_count"),
    F.countDistinct("customer_id").alias("unique_customers_count"),  
)
# Show the aggregated results
order_aggregations.show()

+-------------------+----------------------+
|unique_orders_count|unique_customers_count|
+-------------------+----------------------+
|               1000|                   617|
+-------------------+----------------------+



**What is the order frequency for each customer in the datas?**

In [25]:
from pyspark.sql.functions import count
# Group by 'customer_id' and aggregate with count on 'order_id' to get order frequency
df_order_frequency = df_orders.groupBy('customer_id').agg(count('order_id').alias('order_frequency'))
# Show the results
df_order_frequency.show(30)

+-----------+---------------+
|customer_id|order_frequency|
+-----------+---------------+
|        833|              2|
|        148|              1|
|        463|              2|
|        471|              1|
|        496|              1|
|        737|              2|
|        623|              3|
|        516|              1|
|         85|              1|
|        251|              1|
|        808|              1|
|        883|              1|
|        458|              1|
|        898|              2|
|        588|              2|
|        799|              2|
|        970|              1|
|        472|              2|
|        133|              1|
|        853|              1|
|         78|              2|
|        918|              1|
|        513|              1|
|        974|              1|
|        321|              1|
|        857|              1|
|        362|              1|
|        673|              1|
|        375|              2|
|        876|              1|
+---------


**How many orders were placed on each day in the dataset?**

In [26]:
orders_per_day = df_orders.groupBy("order_date").count().orderBy("order_date")
orders_per_day.show() 

+----------+-----+
|order_date|count|
+----------+-----+
|2021-01-01|    3|
|2021-01-02|    7|
|2021-01-03|    3|
|2021-01-04|    3|
|2021-01-05|    5|
|2021-01-06|    2|
|2021-01-07|    5|
|2021-01-08|    2|
|2021-01-09|    5|
|2021-01-10|    6|
|2021-01-11|    4|
|2021-01-12|    7|
|2021-01-13|    4|
|2021-01-14|    3|
|2021-01-15|    3|
|2021-01-16|    1|
|2021-01-17|    2|
|2021-01-18|    2|
|2021-01-19|    1|
|2021-01-20|    4|
+----------+-----+
only showing top 20 rows



* show sales dataset

In [27]:
file_path = r"C:\Users\abdel\OneDrive\Desktop\Sub\BigData\Csv. files\sales.csv"
df_sales = spark.read.csv(file_path, header=True, inferSchema=True)
df_sales.show()


+--------+--------+----------+--------------+--------+-----------+
|sales_id|order_id|product_id|price_per_unit|quantity|total_price|
+--------+--------+----------+--------------+--------+-----------+
|       0|       1|       218|           106|       2|        212|
|       1|       1|       481|           118|       1|        118|
|       2|       1|         2|            96|       3|        288|
|       3|       1|      1002|           106|       2|        212|
|       4|       1|       691|           113|       3|        339|
|       5|       1|       981|           106|       3|        318|
|       6|       2|       915|            96|       1|         96|
|       7|       2|       686|           113|       1|        113|
|       8|       2|      1091|           115|       3|        345|
|       9|       2|      1196|           105|       1|        105|
|      10|       2|       157|            91|       3|        273|
|      11|       2|      1174|            99|       2|        

In [28]:
df_sales.printSchema()
print("Count of dataframe:",df_sales.count())

root
 |-- sales_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- price_per_unit: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- total_price: integer (nullable = true)

Count of dataframe: 5000


In [29]:
df_sales.describe(["price_per_unit", "quantity", "total_price"]).show()

+-------+-----------------+------------------+-----------------+
|summary|   price_per_unit|          quantity|      total_price|
+-------+-----------------+------------------+-----------------+
|  count|             5000|              5000|             5000|
|   mean|         103.5016|            1.9924|           206.36|
| stddev|9.195004462283432|0.8075101575403906|86.35745666741475|
|    min|               90|                 1|               90|
|    max|              119|                 3|              357|
+-------+-----------------+------------------+-----------------+



# EDA

#### Cleanning data

In [30]:
from pyspark.sql.functions import col
# Handling Missing Values
# Drop rows with any null values
df_sales=df_sales.dropna()

# Handling Duplicates
df_saless =df_sales.dropDuplicates()
print("Count of after cleanning dataframe:",df_sales.count())
df_saless.show()

Count of after cleanning dataframe: 5000
+--------+--------+----------+--------------+--------+-----------+
|sales_id|order_id|product_id|price_per_unit|quantity|total_price|
+--------+--------+----------+--------------+--------+-----------+
|     797|     166|       217|           106|       2|        212|
|     970|     199|      1057|            93|       3|        279|
|    1129|     233|       218|           106|       3|        318|
|    1841|     371|      1013|           106|       1|        106|
|    2058|     415|       487|           118|       2|        236|
|    2085|     419|       581|           100|       2|        200|
|    2145|     432|       191|            98|       3|        294|
|    2474|     501|      1029|            90|       2|        180|
|    2488|     503|       974|            97|       1|         97|
|    2584|     519|       161|            91|       2|        182|
|    2610|     523|       727|           119|       3|        357|
|    2640|     528|  


**What is the average price per unit, total revenue, and total quantity sold?**

In [31]:
from pyspark.sql import functions as F

# Aggregations for df_sales DataFrame
sales_aggregations = df_sales.agg(
    F.countDistinct("sales_id").alias("unique_sales_count"),
    F.countDistinct("order_id").alias("unique_orders_count"),
    F.countDistinct("product_id").alias("unique_products_count"),
    F.avg("price_per_unit").alias("avg_price_per_unit"),
    (F.sum("price_per_unit") * F.sum("quantity")).alias("total_revenue"),
    F.sum("quantity").alias("total_quantity_sold")
)

# Show the aggregated results
sales_aggregations.show()

+------------------+-------------------+---------------------+------------------+-------------+-------------------+
|unique_sales_count|unique_orders_count|unique_products_count|avg_price_per_unit|total_revenue|total_quantity_sold|
+------------------+-------------------+---------------------+------------------+-------------+-------------------+
|              5000|                993|                 1233|          103.5016|   5155414696|               9962|
+------------------+-------------------+---------------------+------------------+-------------+-------------------+





**What is the total sales amount for each product, and which products have the highest total sales?**

In [32]:
sales_by_product = df_sales.groupBy('product_id').sum('total_price').orderBy('sum(total_price)', ascending=False)
sales_by_product.show()

+----------+----------------+
|product_id|sum(total_price)|
+----------+----------------+
|        78|            2832|
|       472|            2714|
|       707|            2499|
|       579|            2400|
|       843|            2373|
|       486|            2360|
|       740|            2289|
|       727|            2261|
|       182|            2254|
|       465|            2124|
|        95|            2124|
|        74|            2124|
|       810|            2106|
|       316|            2071|
|       405|            2023|
|       222|            2014|
|       830|            1989|
|      1188|            1980|
|      1184|            1980|
|      1091|            1955|
+----------+----------------+
only showing top 20 rows



* show products dataset

In [33]:
file_path = r"C:\Users\abdel\OneDrive\Desktop\Sub\BigData\Csv. files\products.csv"
df_products = spark.read.csv(file_path, header=True, inferSchema=True)
# Show the first 10 rows of the DataFrame
df_products.show()

+----------+------------+------------+----+------+-----+--------+--------------------+
|product_ID|product_type|product_name|size|colour|price|quantity|         description|
+----------+------------+------------+----+------+-----+--------+--------------------+
|         0|       Shirt|Oxford Cloth|  XS|   red|  114|      66|A red coloured, X...|
|         1|       Shirt|Oxford Cloth|   S|   red|  114|      53|A red coloured, S...|
|         2|       Shirt|Oxford Cloth|   M|   red|  114|      54|A red coloured, M...|
|         3|       Shirt|Oxford Cloth|   L|   red|  114|      69|A red coloured, L...|
|         4|       Shirt|Oxford Cloth|  XL|   red|  114|      47|A red coloured, X...|
|         5|       Shirt|Oxford Cloth|  XS|orange|  114|      45|A orange coloured...|
|         6|       Shirt|Oxford Cloth|   S|orange|  114|      72|A orange coloured...|
|         7|       Shirt|Oxford Cloth|   M|orange|  114|      77|A orange coloured...|
|         8|       Shirt|Oxford Cloth|   L|

In [34]:
df_products.printSchema()
print("Count of dataframe:",df_products.count())

root
 |-- product_ID: integer (nullable = true)
 |-- product_type: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- size: string (nullable = true)
 |-- colour: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- description: string (nullable = true)

Count of dataframe: 1260


# EDA

#### Cleanning data

In [35]:
# Handling Missing Values
# Drop rows with any null values
df_products= df_products.dropna()

# Handling Duplicates
df_products =df_products.dropDuplicates()
print("Count of after cleanning dataframe:",df_products.count())
df_products.show()

Count of after cleanning dataframe: 1260
+----------+------------+---------------+----+------+-----+--------+--------------------+
|product_ID|product_type|   product_name|size|colour|price|quantity|         description|
+----------+------------+---------------+----+------+-----+--------+--------------------+
|       266|       Shirt|           Polo|   S|  blue|  117|      76|A blue coloured, ...|
|       299|       Shirt|   Cuban Collar|  XL| green|   93|      60|A green coloured,...|
|       439|      Jacket|          Denim|  XL| green|   92|      59|A green coloured,...|
|       458|      Jacket|         Puffer|   L|   red|  110|      44|A red coloured, L...|
|       856|    Trousers|         Chinos|   S| green|  100|      61|A green coloured,...|
|      1032|    Trousers|    Relaxed Leg|   M| green|   95|      43|A green coloured,...|
|      1040|    Trousers|    Relaxed Leg|  XS|indigo|   95|      60|A indigo coloured...|
|      1160|    Trousers|   High-Waisted|  XS|orange|   98|

In [36]:
from pyspark.sql import functions as F

# Aggregations for product DataFrame
product_aggregations = df_products.agg(
    F.countDistinct("product_ID").alias("unique_products_count"),
    F.count("product_type").alias("total_products"),
    F.avg("price").alias("average_price"),
    F.sum("quantity").alias("total_quantity")
)

# Show the aggregated results
product_aggregations.show()

+---------------------+--------------+------------------+--------------+
|unique_products_count|total_products|     average_price|total_quantity|
+---------------------+--------------+------------------+--------------+
|                 1260|          1260|105.80555555555556|         75789|
+---------------------+--------------+------------------+--------------+



In [37]:
# Count of Products by Type
product_type_counts = df_products.groupBy("product_type").count().orderBy("count", ascending=False)
# Show product type counts
product_type_counts.show()
# Statistical Analysis by Product Type (example: average price)
avg_price_by_type = df_products.groupBy("product_type").agg(F.avg("price").alias("avg_price"))
# Show average price by product type
avg_price_by_type.show()

+------------+-----+
|product_type|count|
+------------+-----+
|      Jacket|  420|
|    Trousers|  420|
|       Shirt|  420|
+------------+-----+

+------------+------------------+
|product_type|         avg_price|
+------------+------------------+
|      Jacket|107.41666666666667|
|    Trousers|101.66666666666667|
|       Shirt|108.33333333333333|
+------------+------------------+




**What is the total price for each produ a and how does it vary across different products?**

In [38]:
from pyspark.sql.functions import col
# Calculate total price for each product
df_total_price = df_products.withColumn("total_price", col("quantity") * col("price"))

# Show the DataFrame with total price
df_total_price.show(truncate=False)

+----------+------------+---------------+----+------+-----+--------+--------------------------------------------------+-----------+
|product_ID|product_type|product_name   |size|colour|price|quantity|description                                       |total_price|
+----------+------------+---------------+----+------+-----+--------+--------------------------------------------------+-----------+
|266       |Shirt       |Polo           |S   |blue  |117  |76      |A blue coloured, S sized, Polo Shirt              |8892       |
|299       |Shirt       |Cuban Collar   |XL  |green |93   |60      |A green coloured, XL sized, Cuban Collar Shirt    |5580       |
|439       |Jacket      |Denim          |XL  |green |92   |59      |A green coloured, XL sized, Denim Jacket          |5428       |
|458       |Jacket      |Puffer         |L   |red   |110  |44      |A red coloured, L sized, Puffer Jacket            |4840       |
|856       |Trousers    |Chinos         |S   |green |100  |61      |A green 



**What is the total revenue generated for each product type and color combination in the dataset?** 

In [47]:
# Calculate total price for each product
df_total_price_by_type_color = df_products.withColumn("total_price", col("quantity") * col("price")) \
    .groupBy("product_type", "colour") \
    .agg(sum("total_price").alias("total_price"))

# Show the DataFrame with total price by product type and color
df_total_price_by_type_color.show(truncate=False)


+------------+------+-----------+
|product_type|colour|total_price|
+------------+------+-----------+
|Shirt       |red   |389840     |
|Trousers    |orange|375139     |
|Jacket      |orange|403578     |
|Trousers    |violet|368337     |
|Jacket      |green |389454     |
|Trousers    |blue  |353392     |
|Jacket      |red   |386938     |
|Jacket      |violet|403096     |
|Trousers    |yellow|367561     |
|Shirt       |yellow|399579     |
|Jacket      |yellow|377513     |
|Shirt       |blue  |393531     |
|Shirt       |indigo|392906     |
|Jacket      |blue  |387448     |
|Jacket      |indigo|381438     |
|Shirt       |orange|382718     |
|Trousers    |indigo|348804     |
|Trousers    |red   |381457     |
|Trousers    |green |364529     |
|Shirt       |violet|393587     |
+------------+------+-----------+
only showing top 20 rows



In [48]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, sum, rank

# Define a Window partitioned by product_type and ordered by total_price descending
windowSpec = Window.partitionBy("product_type").orderBy(col("total_price").desc())

# Add a rank column based on total_price within each product_type
df_ranked = df_total_price_by_type_color.withColumn("rank", rank().over(windowSpec))

# Show the DataFrame with total price, product type, color, and rank
df_ranked.show(truncate=False)


+------------+------+-----------+----+
|product_type|colour|total_price|rank|
+------------+------+-----------+----+
|Jacket      |orange|403578     |1   |
|Jacket      |violet|403096     |2   |
|Jacket      |green |389454     |3   |
|Jacket      |blue  |387448     |4   |
|Jacket      |red   |386938     |5   |
|Jacket      |indigo|381438     |6   |
|Jacket      |yellow|377513     |7   |
|Shirt       |yellow|399579     |1   |
|Shirt       |violet|393587     |2   |
|Shirt       |blue  |393531     |3   |
|Shirt       |indigo|392906     |4   |
|Shirt       |red   |389840     |5   |
|Shirt       |green |387925     |6   |
|Shirt       |orange|382718     |7   |
|Trousers    |red   |381457     |1   |
|Trousers    |orange|375139     |2   |
|Trousers    |violet|368337     |3   |
|Trousers    |yellow|367561     |4   |
|Trousers    |green |364529     |5   |
|Trousers    |blue  |353392     |6   |
+------------+------+-----------+----+
only showing top 20 rows



#### Perform the join operation

In [40]:
# Assuming df_sales is your DataFrame with sales data
# Calculate the total amount spent per order ID
df_total_amount = df_sales.withColumn("total_amount", col("price_per_unit") * col("quantity")) \
    .groupBy("order_id") \
    .agg(sum("total_amount").alias("total_amount_per_order")) \
    .orderBy("order_id")

# Show the total amount spent per order ID
df_total_amount.show(truncate=False)


+--------+----------------------+
|order_id|total_amount_per_order|
+--------+----------------------+
|1       |1487                  |
|2       |1130                  |
|3       |508                   |
|4       |976                   |
|5       |2043                  |
|6       |732                   |
|7       |523                   |
|8       |299                   |
|9       |1315                  |
|10      |874                   |
|11      |461                   |
|12      |714                   |
|13      |1583                  |
|14      |1254                  |
|15      |1365                  |
|16      |1786                  |
|17      |670                   |
|18      |952                   |
|19      |1244                  |
|20      |1557                  |
+--------+----------------------+
only showing top 20 rows



In [41]:

# Perform the join operation
df_merged = df_orders.join(df_total_amount, on='order_id', how='left')

# Sort the merged DataFrame by the 'total_amount' column in descending order
df_merged_sorted = df_merged.orderBy('total_amount_per_order', ascending=False)

# Show the sorted DataFrame
df_merged_sorted.show(truncate=False)


+--------+-----------+-------+----------+-------------+----------------------+
|order_id|customer_id|payment|order_date|delivery_date|total_amount_per_order|
+--------+-----------+-------+----------+-------------+----------------------+
|77      |680        |56675  |2021-04-19|2021-04-29   |2733                  |
|252     |683        |11465  |2021-01-11|2021-02-05   |2597                  |
|966     |639        |28100  |2021-02-25|2021-03-10   |2550                  |
|668     |22         |40483  |2021-03-15|2021-03-30   |2539                  |
|705     |107        |11686  |2021-01-14|2021-01-25   |2495                  |
|142     |237        |53165  |2021-06-15|2021-07-12   |2469                  |
|734     |623        |30153  |2021-01-25|2021-01-31   |2455                  |
|530     |467        |20275  |2021-01-07|2021-01-23   |2450                  |
|970     |382        |36941  |2021-01-04|2021-01-23   |2390                  |
|936     |148        |47197  |2021-07-16|2021-08-01 

In [42]:
# Perform the join operation using customer_id
df_merged2 = df_merged.join(df_customers, on='customer_id', how='left')

# Show the merged DataFrame
df_merged2.show(truncate=False)

+-----------+--------+-------+----------+-------------+----------------------+--------------------+------+---+-----------------------------+--------+------------------+----------------------------+
|customer_id|order_id|payment|order_date|delivery_date|total_amount_per_order|customer_name       |gender|age|home_address                 |zip_code|city              |state                       |
+-----------+--------+-------+----------+-------------+----------------------+--------------------+------+---+-----------------------------+--------+------------------+----------------------------+
|873        |55      |55650  |2021-04-08|2021-04-25   |119                   |Cecilla Pigford     |Other |38 |5538 Jett MallSuite 854      |5510    |Samanthatown      |Western Australia           |
|631        |66      |42774  |2021-08-14|2021-08-22   |2077                  |Wanids Cudby        |Female|49 |80 Chelsea KnollApt. 014     |7749    |Davidfurt         |Northern Territory          |
|366      

In [43]:
from pyspark.sql.functions import desc
# Select specific columns to show
df_selected = df_merged2.select("customer_id", "customer_name", "total_amount_per_order")
# Sort the DataFrame by 'total_amount_per_order' column in descending order
df_sorted = df_selected.orderBy(desc('total_amount_per_order'))
# Show the sorted DataFrame
df_sorted.show(truncate=False)

+-----------+----------------------+----------------------+
|customer_id|customer_name         |total_amount_per_order|
+-----------+----------------------+----------------------+
|680        |Camellia Heatherington|2733                  |
|683        |Esteban Hensmans      |2597                  |
|639        |Tobin Jeffryes        |2550                  |
|22         |Blayne Sabatier       |2539                  |
|107        |Sandy Yushachkov      |2495                  |
|237        |Cassaundra Sliman     |2469                  |
|623        |Hort Ridding          |2455                  |
|467        |Cacilie Forrestill    |2450                  |
|382        |Emanuele Deesly       |2390                  |
|148        |Ikey Spohr            |2376                  |
|282        |Wren Helgass          |2333                  |
|565        |Hoebart Edger         |2316                  |
|439        |Daren Airs            |2300                  |
|797        |Rachelle Jelf         |2298